In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchtext.vocab import GloVe
import spacy
import io
import jsonlines
import json
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import random_split
import fasttext
import re
from spacy.language import Language
from spacy.tokens import Doc
from gensim import models
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np



cpu = torch.device('cpu')


use_word2vec = False
use_glove = True
use_fasttext = False


if torch.has_mps:
    device = torch.device('mps')
else:
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')


EMBED_DIM = 300
HIDDEN_LAYER_DIM = 100
NUM_EPOCHS = 40
EXPAND_CONTRACTIONS = False

In [3]:
def expand_contractions_text(text):    
    flags = re.IGNORECASE | re.MULTILINE
    text = re.sub(r'`', "'", text, flags = flags)
    ## starts / ends with '
    text = re.sub(
        r"(\s|^)'(aight|cause)(\s|$)",
        '\g<1>\g<2>\g<3>',
        text, flags = flags
    )
    text = re.sub(
        r"(\s|^)'t(was|is)(\s|$)", r'\g<1>it \g<2>\g<3>',
        text,
        flags = flags
    )
    text = re.sub(
        r"(\s|^)ol'(\s|$)",
        '\g<1>old\g<2>',
        text, flags = flags
    )
        
    text = re.sub(r"\b(aight)\b", 'alright', text, flags = flags)
    text = re.sub(r'\bcause\b', 'because', text, flags = flags)
    text = re.sub(r'\b(finna|gonna)\b', 'going to', text, flags = flags)
    text = re.sub(r'\bgimme\b', 'give me', text, flags = flags)
    text = re.sub(r"\bgive'n\b", 'given', text, flags = flags)
    text = re.sub(r"\bhowdy\b", 'how do you do', text, flags = flags)
    text = re.sub(r"\bgotta\b", 'got to', text, flags = flags)
    text = re.sub(r"\binnit\b", 'is it not', text, flags = flags)
    text = re.sub(r"\b(can)(not)\b", r'\g<1> \g<2>', text, flags = flags)
    text = re.sub(r"\bwanna\b", 'want to', text, flags = flags)
    text = re.sub(r"\bmethinks\b", 'me thinks', text, flags = flags)
    text = re.sub(r"\bo'er\b", r'over', text, flags = flags)
    text = re.sub(r"\bne'er\b", r'never', text, flags = flags)
    text = re.sub(r"\bo'?clock\b", 'of the clock', text, flags = flags)
    text = re.sub(r"\bma'am\b", 'madam', text, flags = flags)
    text = re.sub(r"\bgiv'n\b", 'given', text, flags = flags)
    text = re.sub(r"\be'er\b", 'ever', text, flags = flags)
    text = re.sub(r"\bd'ye\b", 'do you', text, flags = flags)
    text = re.sub(r"\be'er\b", 'ever', text, flags = flags)
    text = re.sub(r"\bd'ye\b", 'do you', text, flags = flags)
    text = re.sub(r"\bg'?day\b", 'good day', text, flags = flags)
    text = re.sub(r"\b(ain|amn)'?t\b", 'am not', text, flags = flags)
    text = re.sub(r"\b(are|can)'?t\b", r'\g<1> not', text, flags = flags)
    text = re.sub(r"\b(let)'?s\b", r'\g<1> us', text, flags = flags)
    text = re.sub(r"\by'all'dn't've'd\b", 'you all would not have had', text, flags = flags)
    text = re.sub(r"\by'all're\b", 'you all are', text, flags = flags)
    text = re.sub(r"\by'all'd've\b", 'you all would have', text, flags = flags)
    text = re.sub(r"(\s)y'all(\s)", r'\g<1>you all\g<2>', text, flags = flags)  
    text = re.sub(r"\b(won)'?t\b", 'will not', text, flags = flags)
    text = re.sub(r"\bhe'd\b", 'he had', text, flags = flags)
    text = re.sub(r"\b(I|we|who)'?d'?ve\b", r'\g<1> would have', text, flags = flags)
    text = re.sub(r"\b(could|would|must|should|would)n'?t'?ve\b", r'\g<1> not have', text, flags = flags)
    text = re.sub(r"\b(he)'?dn'?t'?ve'?d\b", r'\g<1> would not have had', text, flags = flags)
    text = re.sub(r"\b(daren|daresn|dasn)'?t", 'dare not', text, flags = flags)
    text = re.sub(r"\b(he|how|i|it|she|that|there|these|they|we|what|where|which|who|you)'?ll\b", r'\g<1> will', text, flags = flags)
    text = re.sub(r"\b(everybody|everyone|he|how|it|she|somebody|someone|something|that|there|this|what|when|where|which|who|why)'?s\b", r'\g<1> is', text, flags = flags)
    text = re.sub(r"\b(I)'?m'a\b", r'\g<1> am about to', text, flags = flags)
    text = re.sub(r"\b(I)'?m'o\b", r'\g<1> am going to', text, flags = flags)
    text = re.sub(r"\b(I)'?m\b", r'\g<1> am', text, flags = flags)
    text = re.sub(r"\bshan't\b", 'shall not', text, flags = flags)
    text = re.sub(r"\b(are|could|did|does|do|go|had|has|have|is|may|might|must|need|ought|shall|should|was|were|would)n'?t\b", r'\g<1> not', text, flags = flags)
    text = re.sub(r"\b(could|had|he|i|may|might|must|should|these|they|those|to|we|what|where|which|who|would|you)'?ve\b", r'\g<1> have', text, flags = flags)
    text = re.sub(r"\b(how|so|that|there|these|they|those|we|what|where|which|who|why|you)'?re\b", r'\g<1> are', text, flags = flags)
    text = re.sub(r"\b(I|it|she|that|there|they|we|which|you)'?d\b", r'\g<1> had', text, flags = flags)
    text = re.sub(r"\b(how|what|where|who|why)'?d\b", r'\g<1> did', text, flags = flags)    
    return text


class ExpandContractionsClass:
    def __init__(self, nlp: Language):
        self.nlp = nlp
    
    def __call__(self,doc: Doc):
        text = doc.text
        print(text)
        return self.nlp.make_doc(expand_contractions_text(text))
    
@Language.factory("expand_contractions_component")
def create_expand_contractions_component(nlp : Language, name: str):
    return ExpandContractionsClass(nlp)
 

In [4]:
nlp = spacy.load("en_core_web_sm")
if EXPAND_CONTRACTIONS:
    nlp.add_pipe("expand_contractions_component",before='tagger')


In [6]:


def preprocess_text(text):    
    words = nlp(text)
    #sentence = [token.text_with_ws for token in words if not token.is_punct]
    sentence  = "".join([token.text_with_ws for token in words if not token.is_punct ]).strip()
    sentence = sentence.lower()
    return sentence

def process_training_data():
    negative_reviews = io.open('./Train.neg',encoding='latin-1').readlines()
    positive_reviews = io.open('./Train.pos',encoding='latin-1').readlines()
    with jsonlines.open('train.jsonl',mode='w') as writer:

        for review in positive_reviews:
            processed_text = preprocess_text(review)
            d = {'text': processed_text , 'sentiment': 1}
            writer.write(d)
        for review in negative_reviews:
            processed_text = preprocess_text(review)
            d = {'text': processed_text , 'sentiment': 0}
            writer.write(d)
process_training_data()


In [7]:
f = open("./train.jsonl",'r')
for line in f.readlines():
    for word in line.split():
        if "\'" in word:
            print(word)

century's
he's
jackson's
tolkien's
that's
doesn't
it's
movie's
it's
it's
stevenson's
there's
it's
it's
vincent's
that's
don't
50's
johnson's
you'd
"you'd
you're
derrida's
1998's
mother's
gollum's
you're
gollum's
can't
"it's
year's
there's
that's
movie's
damon's
film's
early-'80s
it's
mcfarlane's
it's
it's
irwin's
it's
piesiewicz's
kieslowski's
it's
it's
today's
earnhart's
"it's
it's
it's
it's
"it's
don't
fessenden's
"it's
"it's
haven't
you're
"it's
doesn't
"it's
he's
"here's
"it's
don't
yvan's
year's
behan's
it's
it'sworth
book's
earnhart's
you'll
it's
"there's
iran's
"it's
it's
it's
isn't
you'll
haneke's
nicholson'scareer",
don't
i'm
you're
you'll
won't
it's
it's
film's
doesn't
couldn't
"moore's
1995's
can't
amy's
amy's
o's
polanski's
petin's
can't
he'd
hubac's
"moore's
that's
doesn't
you'll
wouldn't
it's
it's
it's
jie's
america's
what's
"it's
it's
wilde's
"münch's
film's
life's
you're
it's
it's
"it's
movie's
don't
"what's
you're
kinnear's
man's
america's
behan's
film's
it's
"it's
don

In [ ]:
"""w = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin',binary=True)
w.get_vector('rat')"""

In [8]:
from torch.utils.data import DataLoader,Dataset

class ReviewDataSet(Dataset):

    def __init__(self,file):
        super().__init__()
        
        self.file = file
        self.data = []
        with open(self.file) as f:
            for line in f:
                sample = json.loads(line)
                self.data.append([sample['text'],sample['sentiment']])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

    

dataset = ReviewDataSet('train.jsonl') 

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset,validation_dataset = random_split(dataset,[train_size,test_size])

train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True)
val_dataloader = DataLoader(validation_dataset,batch_size=32,shuffle=True)

In [13]:
def getWordEmbeddings(batch_data,dicts):
    
    word2vec,glove,fasttext = dicts

    if(len(batch_data)==2):
        reviews = batch_data[0]
        sentiment = batch_data[1]
    else:
        reviews = batch_data[0]
        sentiment = None
    
    reviews_tensor = []
    lengths = []
    for review in reviews:
        words = review.split()
        words_tensor = []
        lengths.append(len(words))
        for word in words:
            #words_tensor.append(glove.__getitem__(word))
            word_embeddings = []
            if use_word2vec:
                pass
            if use_glove:
                word_embeddings.append(glove.__getitem__(word))
            if use_fasttext:
                pass
            word_tensor = torch.stack(word_embeddings)
            word_tensor = word_tensor.squeeze()
            words_tensor.append(word_tensor)            
        reviews_tensor.append(torch.stack(words_tensor,dim=0))
    
    mask = torch.zeros((len(lengths),max(lengths)))
    for i in range(len(lengths)):
        mask[i,:lengths[i]] = 1.0
    
    return (pad_sequence(reviews_tensor,batch_first=True),mask,sentiment)




In [ ]:
'''from gensim import models

w = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin',binary=True)
w.get_vector('rat')
'''

In [14]:
class DAN(nn.Module):

    def __init__(self,embed_dim=EMBED_DIM,hidden_dim = HIDDEN_LAYER_DIM):
        super().__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.fc1 = nn.Linear(self.embed_dim,self.hidden_dim)
        self.fc = nn.Linear(self.hidden_dim,self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim,1)
        self.sigmoid  = nn.Sigmoid()

    def forward(self,inp,inp_mask):
        
        inp_lengths = torch.sum(inp_mask,-1,keepdim=True)
        total = torch.sum(inp*(inp_mask.unsqueeze(2)),axis=1)
        vector_average = total / inp_lengths
        ans = F.relu(self.fc1(vector_average))
        ans = F.relu(self.fc(ans))
        ans = self.sigmoid(self.fc2(ans))
        return ans
    


In [15]:
'''inp = torch.randn((2,3,5))
mask = torch.tensor([[1.0,0.0,0.0],[1.0,0.0,1.0]])

print(inp)'''

'inp = torch.randn((2,3,5))\nmask = torch.tensor([[1.0,0.0,0.0],[1.0,0.0,1.0]])\n\nprint(inp)'

In [11]:
glove = GloVe()
fasttext_model = fasttext.load_model('./crawl-300d-2M-subword/crawl-300d-2M-subword.bin')
word2vec_model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin',binary=True)

In [16]:
# Training loop
dan = DAN(EMBED_DIM,HIDDEN_LAYER_DIM)

criterion = nn.BCELoss()
optimizer = optim.Adam(dan.parameters(),lr=0.01)

for e in range(NUM_EPOCHS):
    training_loss = 0.0
    size = 0
    dan.train()

    for i,data in enumerate(train_dataloader,0):
        
        optimizer.zero_grad()
        
        input_reviews , input_mask , output_labels = getWordEmbeddings(data,(None,glove,None))

        output = dan(input_reviews,input_mask).squeeze()

        loss = criterion(output,output_labels.float())

        training_loss += loss.item()
        loss.backward()
        optimizer.step()
        size = max(size,i+1)

    dan.eval()
    validation_loss = 0

    val_size = 0
    for i,data in enumerate(val_dataloader,0):
        input_reviews,input_mask,output_labels = getWordEmbeddings(data,(None,glove,None))
        output = dan(input_reviews,input_mask).squeeze()
        nearest_class = torch.round(output)
        correct = (nearest_class == output_labels.float()).float()
        validation_loss += correct.sum()
  
    print(str(training_loss/size )+ "   " + str(validation_loss/len(validation_dataset)))


0.507449073433876   tensor(0.7210)
0.4709497238397598   tensor(0.7375)
0.4505024184584618   tensor(0.7555)
0.43656498450040815   tensor(0.7605)
0.42873722803592684   tensor(0.7620)
0.41367686998844144   tensor(0.7285)
0.3981266605257988   tensor(0.7590)
0.3810367125272751   tensor(0.7635)
0.36195875442028047   tensor(0.7635)
0.3460372394621372   tensor(0.7480)
0.325171503841877   tensor(0.7480)
0.3193856512308121   tensor(0.7380)
0.29741918992996214   tensor(0.7415)
0.2753442927598953   tensor(0.7340)
0.2667115894556046   tensor(0.7395)
0.2670583936870098   tensor(0.7270)
0.232670334354043   tensor(0.7295)
0.21992873398959636   tensor(0.7335)
0.21117843835055827   tensor(0.7160)
0.20935773608833552   tensor(0.7300)
0.18699572509527207   tensor(0.7300)
0.17573961452394724   tensor(0.7240)
0.17244292141497136   tensor(0.7190)
0.16455344831198454   tensor(0.7075)
0.17485630059614776   tensor(0.7305)
0.14938848882168532   tensor(0.7240)
0.14777845878899099   tensor(0.7275)
0.14408338870108

In [20]:
def test(filename):
    reviews = open(filename,'r').readlines()
    for i in range(len(reviews)):
        r = reviews[i]
        reviews[i] = preprocess_text(r)
    
    reviews,reviews_mask,labels = getWordEmbeddings([reviews],(None,glove,None))
    dan.eval()

    output = dan(reviews,reviews_mask)
    print(output)


test('test.txt')


tensor([[0.0000],
        [1.0000],
        [1.0000],
        [0.0020]], grad_fn=<SigmoidBackward0>)
